In [60]:
from typing import Dict, Text
import pprint

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

import pandas as pd
 
# creating a data frame
recipes = pd.read_csv("recipes.csv")
ratings = pd.read_csv("ratings.csv")

recipes = recipes.astype({"recipeId": str})
ratings = ratings.astype({"userId": str, "recipeId": str, "rating": str})

newColumn = []


# for i in recipes['categories']:
#     temp = i.split(" | ")
#     newColumn.append(temp)

# recipes.drop(columns=['categories'])
# recipes["categories"] = newColumn
# print(recipes)

# print(recipes.dtypes)

recipes = tf.data.Dataset.from_tensor_slices(dict(recipes))
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings))

ratings = ratings.map(lambda x: {
    "name": x["name"],
    "userId": x["userId"]
})

recipes = recipes.map(lambda x: x["name"])
                      
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["userId"]))

recipe_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
recipe_titles_vocabulary.adapt(recipes)


class RecipeModel(tfrs.Model):

  def __init__(
      self,
      user_model: tf.keras.Model,
      recipe_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    self.user_model = user_model
    self.recipe_model = recipe_model
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    user_embeddings = self.user_model(features["userId"])
    positive_recipe_embeddings = self.recipe_model(features["name"])
    return self.task(user_embeddings, positive_recipe_embeddings)

user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
recipe_model = tf.keras.Sequential([
    recipe_titles_vocabulary,
    tf.keras.layers.Embedding(recipe_titles_vocabulary.vocab_size(), 64)
])

task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    recipes.batch(128).map(recipe_model)
  )
)

model = RecipeModel(user_model, recipe_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

model.fit(ratings.batch(4096), epochs=5)

index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    recipes.batch(100).map(lambda title: (title, model.recipe_model(title))))

# Get some recommendations.

{'name': b'4Z48C3WAYR', 'userId': b'5408'}
Epoch 1/5
3/3 [==============================] - 4s 818ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 6.0006e-04 - factorized_top_k/top_100_categorical_accuracy: 0.0022 - loss: 23813.1938 - regularization_loss: 0.0000e+00 - total_loss: 23813.1938
Epoch 2/5
3/3 [==============================] - 3s 803ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0047 - factorized_top_k/top_5_categorical_accuracy: 0.0505 - factorized_top_k/top_10_categorical_accuracy: 0.0658 - factorized_top_k/top_50_categorical_accuracy: 0.1606 - factorized_top_k/top_100_categorical_accuracy: 0.2398 - loss: 21729.4966 - regularization_loss: 0.0000e+00 - total_loss: 21729.4966  
Epoch 3/5
3/3 [==============================] - 3s 753ms/step - factorized_top_k/top_1_categorical_accuracy:

In [61]:
_, titles = index(np.array(["10"]))
print(f"Top 3 recommendations for user 10: {titles[0, :3]}")

Top 3 recommendations for user 10: [b'CFZG' b'WR' b'WR']
